In [1]:
import os
import numpy as np
import pandas as pd
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [2]:
data = pd.read_csv('s3://group2-code/heart.csv')

In [3]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Separate them to [X, Y]. Where our target labels are 'Y', and 'X' is our training data.
Y = data.target.values
X = data.drop(['target'], axis=1)

# Now split to train/test with 80% training data, and 20% test data.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Check dimensions of both sets.
print("Train Features Size:", X_train.shape)
print("Test Features Size:", X_test.shape)
print("Train Labels Size:", Y_train.shape)
print("Test Labels Size:", Y_test.shape)

Train Features Size: (242, 13)
Test Features Size: (61, 13)
Train Labels Size: (242,)
Test Labels Size: (61,)


In [5]:
def NN_model(learning_rate):
    model = Sequential()
    model.add(Dense(32, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, activation='softmax'))
    Adam(lr=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [6]:
# Build a NN-model, and start training
learning_rate = 0.01
model = NN_model(learning_rate)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                448       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=16, verbose=2)

Epoch 1/10
16/16 - 0s - loss: 1.0792 - accuracy: 0.5331 - val_loss: 0.8952 - val_accuracy: 0.5246
Epoch 2/10
16/16 - 0s - loss: 0.7806 - accuracy: 0.5537 - val_loss: 0.7392 - val_accuracy: 0.4754
Epoch 3/10
16/16 - 0s - loss: 0.6870 - accuracy: 0.5331 - val_loss: 0.5957 - val_accuracy: 0.7869
Epoch 4/10
16/16 - 0s - loss: 0.6624 - accuracy: 0.6074 - val_loss: 0.5938 - val_accuracy: 0.6230
Epoch 5/10
16/16 - 0s - loss: 0.6486 - accuracy: 0.6240 - val_loss: 0.6090 - val_accuracy: 0.6230
Epoch 6/10
16/16 - 0s - loss: 0.6187 - accuracy: 0.6694 - val_loss: 0.5520 - val_accuracy: 0.8033
Epoch 7/10
16/16 - 0s - loss: 0.6145 - accuracy: 0.6364 - val_loss: 0.5463 - val_accuracy: 0.6885
Epoch 8/10
16/16 - 0s - loss: 0.5937 - accuracy: 0.6901 - val_loss: 0.5295 - val_accuracy: 0.8033
Epoch 9/10
16/16 - 0s - loss: 0.5894 - accuracy: 0.6694 - val_loss: 0.5250 - val_accuracy: 0.8197
Epoch 10/10
16/16 - 0s - loss: 0.6021 - accuracy: 0.6818 - val_loss: 0.5364 - val_accuracy: 0.7541


In [8]:
predictions = np.argmax(model.predict(X_test), axis=1)
model_accuracy = accuracy_score(Y_test, predictions)*100
print("Model Accracy:", model_accuracy,"%")
print(classification_report(Y_test, predictions))

Model Accracy: 75.40983606557377 %
              precision    recall  f1-score   support

           0       0.68      0.93      0.78        29
           1       0.90      0.59      0.72        32

    accuracy                           0.75        61
   macro avg       0.79      0.76      0.75        61
weighted avg       0.80      0.75      0.75        61



In [9]:
print('END OF NOTEBOOK')

END OF NOTEBOOK
